# **Thư viện**

In [ ]:
import pandas as pd
import numpy as np
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


# **Dữ liệu**

In [3]:
df = pd.read_csv("../data/raw/EUR_VND_Exchange_Rate.csv")
df.head()

,timestamp,open,high,low,close
0,2025-04-01,27649.0,27703.0,27622.0,27650.0
1,2025-03-31,27675.0,27675.0,27570.0,27646.0
2,2025-03-28,27588.0,27670.0,27515.0,27658.0
3,2025-03-27,27508.0,27603.0,27505.0,27592.0
4,2025-03-26,27612.0,27618.0,27451.0,27469.0


In [1]:
X_train, y_train, time_train, X_test, y_test, time_test = load_and_preprocess_data(
    "../data/raw/EUR_VND_Exchange_Rate.csv", seq_length=30
)
print(len(X_train), len(X_test))

NameError: name 'load_and_preprocess_data' is not defined

In [ ]:
X_train

# **Mô hình LSTM**

In [ ]:
def build_lstm_model(hp, seq_length):
    """Xây dựng mô hình LSTM với hyperparameter tuning."""
    model = Sequential()
    model.add(Input(shape=(seq_length, 1)))

    model.add(LSTM(
        units=hp.Int("lstm_units_1", 32, 128, 16),
        return_sequences=True,
        activation="tanh",
        kernel_regularizer=l2(hp.Choice("l2_reg", [0.001, 0.01, 0.1]))
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float("dropout_1", 0.1, 0.5, 0.1)))

    model.add(LSTM(
        units=hp.Int("lstm_units_2", 32, 128, 16),
        return_sequences=False,
        activation="tanh",
        kernel_regularizer=l2(hp.Choice("l2_reg", [0.001, 0.01, 0.1]))
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float("dropout_2", 0.1, 0.5, 0.1)))

    model.add(Dense(hp.Int("dense_units", 16, 64, 16), activation="relu"))
    model.add(Dense(1))

    model.compile(
        optimizer=Adam(learning_rate=hp.Choice("learning_rate", [0.001, 0.0005, 0.0001])),
        loss="mse",
        metrics=["mae"]
    )

    return model

def train_lstm(X_train, y_train, X_test, y_test, seq_length, model_path):
    """Huấn luyện và lưu mô hình LSTM sử dụng hyperparameter tuning."""
    tuner = kt.RandomSearch(
        lambda hp: build_lstm_model(hp, seq_length),
        objective="val_loss",
        max_trials=10,
        executions_per_trial=2,
        directory=r"C:\Users\DELL\Downloads\KÌ 2 (2024 - 2025)\Phân tích chuỗi thời gian\Project\models",
    )

    early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(model_path, save_best_only=True)

    tuner.search(X_train, y_train,
                 epochs=50,
                 batch_size=32,
                 validation_data=(X_test, y_test),
                 callbacks=[early_stopping, model_checkpoint])

    return tuner.get_best_hyperparameters(num_trials=1)[0]


In [ ]:
best_hp = train_lstm(
    X_train, y_train, 
    X_test, y_test, 
    seq_length=30, 
    model_path=r"C:\Users\DELL\Downloads\KÌ 2 (2024 - 2025)\Phân tích chuỗi thời gian\Project\models\best_lstm.h5", 
)

In [ ]:
best_hp.values